# Homework 4

In this homework you will get experience with Q-learning applied to some classic domains from the early literature on reinforcement learning.  You'll implement tabular Q-learning, in which the states and actions must be discrete.  The underlying domains have discrete action spaces but continuous observation spaces.  I've provided code that will convert continuous observations into discrete ones.  In a later homework we'll use neural networks to solve these same problems without the need to discretize.

## Task 1: Set up your environment

<b>There is nothing to turn in for this task.</b>

You'll need to pip install the following packages:
* gymnasium[classic-control] - a collection of RL domains
* tqdm - a tool for monitoring the progress of loops that run a long time
* numpy - a collection of useful tools for "mathy" things
* matplotlib - a collection of plotting utilities

In [ ]:
import gymnasium as gym
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

## Task 2: Look at the gymnasium documentation

<b>There is nothing to turn in for this task.</b>

Gymnasium is a package that has a uniform interface to a variety of domains where RL can be used.  If your code works for one of them, it will (in theory) work for all of them.  The top-level documentation is here:

https://gymnasium.farama.org/index.html

We'll work with 3 domains.  Read the documentation for each of them:
* Mountain car - https://gymnasium.farama.org/environments/classic_control/mountain_car/
* Acrobot - https://gymnasium.farama.org/environments/classic_control/acrobot/
* Lunar lander - https://gymnasium.farama.org/environments/box2d/lunar_lander/


Each of the domains produces observations that are vectors of real values.  For example, as the documentation for the Mountain Car domain says the state has two real values:
* The position of the car on the x axis
* The velocity of the car

The class below converts real-valued vectors into discrete values.  You will experiment with the impacts of using coarse or fine discretization.  To turn a given observation that is a real-valued vector into a discrete state, the class below divides the range of each variable into a set of uniformly sized, non-overlapping bins.

For example, for the Mountain Car the smallest and largest values of x are -1.2 and 0.6, respectively.  If you select 5 bins, the size of each bin will be (0.6 - -1.2)/5 = 0.36.  They span the following ranges, which get mapped to distinct integers as shown below:
* [-1.2, -0.84) -> 0
* [-0.84, -0.48) -> 1
* [-0.48, -0.12) -> 2
* [-0.12, 0.24) -> 3
* [0.24, 0.6] -> 4

Each element of an observation gets mapped like this, and the resulting string of digits becomes a key to map to the corresponding discrete state.  Each time a new key is found (i.e., the system finds itself in a discrete state that it has never seen before) it is mapped to an integer that can be used to index into the Q-table.

Note that when the number of bins is small, the system treats lots of underlying observations as the same state.  When the number of bins is large, the system can make more fine distinctions but the Q-table gets to be large and you'll need more experience in the domain to learn about all of those states.  You will explore that tradeoff below.

In [ ]:
# Let's look at an observation in the mountain car domain
env = gym.make("MountainCar-v0", render_mode=None)
observation, info = env.reset()
observation

In [ ]:
class Discrete:
    
    def __init__(self, env, nbins):
        """
        Arguments:
          env - A Gymnasium environment that was created by a call to gym.make()
          nbins - If this is an integer, then each of the elements of an observation
                  are mapped into nbins non-overlapping intervals whose size is
                  (high - low) / nbins.  If this is a list, then the list must be the
                  same size as an observation and each element of the list specifies the
                  number of bins for the corresponding element of an observation.
                  This makes it possible to use different numbers of bins for 
                  different elements of an observation.
        """
        
        nobs = env.observation_space.shape[0]
        if type(nbins) == int:
            nbins = [nbins] * nobs
        else:
            assert len(nbins) == nobs, "You must supply %d bin values" % nobs
        self.env = env
        self.nbins = nbins
        self.widths = []
        for low, high, n in zip(env.observation_space.low,
                                env.observation_space.high,
                                nbins):
            self.widths.append((high-low)/n)

        self.state_map = {}

    
    def size(self):
        """
        Return the size of the state space.
        """
        
        return np.prod(self.nbins)

    
    def discretize(self, obs):
        """
        Return the discrete state to which an observation corresponds.
        """
        
        state = '' 
        for value, low, width in zip(obs, env.observation_space.low, self.widths):
            state += '%d' % ((value - low)/width)
        if state not in self.state_map:
            self.state_map[state] = len(self.state_map)
        return self.state_map[state]

## Task 3: Experiment with different discretization granularities

<b>There is nothing to turn in for this task.</b>

Below is an example of running the Mountain Car environment for a few steps and printing out the observation and state.  Note what happens when the number of bins is 10 in terms of which states are visited.  Change it to other values, higher and lower, and see how the states change in terms of granularity.

In [ ]:
disc = Discrete(env, 10)  # <--- Change the 10 to other values and explore

print("Number of distinct states = %d" % disc.size())

for _ in range(50):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    state = disc.discretize(observation)
    print ('State = %s, Observation = %s' % (state, observation))

## Task 4: Finish the Q-learner

<b>The code that you write for this task will be part of your grade on this assignment.</b>

Below is a Q-learner class.  It has an init() method and a method for choosing the greedy action.  You'll need to
* add a method for choosing an epsilon-greedy action
* add a method for performing a Q update

I've provided stubs for those methods.  Recall that the epsilon-greedy action is one that is randomly chosen with probability epsilon and greedy with probability 1 - epsilon.

In [ ]:
class Q:

    def __init__(self, nstates, nactions):
        """
        Arguments:
          nstates - The number of distinct states
          nactions - The number of distinct actions
        """
        
        self.gamma = 0.999      # Discount factor
        self.alpha = 0.1        # Learning rate
        self.epsilon = 0.05     # Exploration probability

        # Create a Q-table initialized to 0
        self.table = np.zeros((nstates, nactions))

    
    def greedy_action(self, state):
        """
        Return the greedy action for a state.  If multiple actions have the 
        same highest Q-value, choose one of them at random.

        Arguments:
          state - The state in which the action is to be taken

        Returns: The optimal action
        """
        
        qmax = self.table[state].max()
        greedy = [idx for idx, value in enumerate(self.table[state]) if value == qmax]
        return random.choice(greedy)


    ###
    ### You need to write this
    ###
    def get_action(self, state):
        """
        Choose an action that is epsilon greedy in a given state.

        Arguments:
          state - The state in which the action is to be taken

        Returns: The action

        """
        
        pass
        

    ###
    ### You need to write this
    ###
    def update(self, state1, action, reward, state2):
        """
        Given that an action was taken in state 1, leading to a specific reward and
        a transition to state 2, perform one update on the Q-table.
        """
        
        pass                                                 

## Task 5: Watch a domain run

<b>There is nothing to turn in for this task.</b>

The function below allows you to run a domain using a Q-table for action selection and see what is going on.  Try calling it with each of the domains below to see them in action.  Note that the Q-table is initialized to all zeros so the greedy action is random.

When you run the function you should see a window pop up with a visualization of the domain.

In [ ]:
MOUNTAIN_CAR = "MountainCar-v0"
ACROBOT = "Acrobot-v1"
LUNAR_LANDER = "LunarLander-v2"

In [ ]:
def run_domain(env, q, disc, steps):
    """
    Arguments:
      env - A Gymnasium enviroment that was created with gym.make()
      q - A Q instance
      disc - A Discrete instance
      steps - The number of steps for which to run the domain
    """
    
    observation, info = env.reset()

    for _ in tqdm(range(steps)):
        state = disc.discretize(observation)
        action = q.greedy_action(state)  
        observation, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated:
            observation, info = env.reset()

In [ ]:
# Create a domain - NOTE that using render_mode "human" visualizes the domain
env = gym.make(MOUNTAIN_CAR, render_mode="human")

# Create an object to discretize it
disc = Discrete(env, 10)

# Create a Q-learner
q = Q(disc.size(), env.action_space.n)

# Run the domain
run_domain(env, q, disc, 500)

## Task 6: Implement Q-learning and test it on the Mountain Car domain

<b>Your code for Q-learning will be part of your grade for this homework.</b>

The Mountain Car domain is the easiest one so you should start there.  I've found that using the default parameters in the Q class, nbins = 30, and 500K steps you can learn an optimal policy.

I've written a stub for the Q-learning function below that you can fill in.

Things to keep in mind:
* During training you want render_mode = None or it will be very slow
* If a step() in the domain makes terminated or truncated true, that means the episode ended and you need to reset() the domain.  You can look at run_domain() above to see how I handle that.

In [ ]:
def learn_domain(env, q, disc, steps):
    """
    Arguments:
      env - A Gymnasium enviroment that was created with gym.make()
      q - A Q instance
      disc - A Discrete instance
      steps - The number of steps for which to run the domain and perform Q updates
    """

    pass

The two cells below, if your Q-learner and training function are correct, will yield optimal behavior in the Mountain Car domain.

In [ ]:
# Create a Mountain Car with render_mode = None so that it runs fast
env = gym.make(MOUNTAIN_CAR, render_mode=None)

# Use 30 bins for discretition of each element of the observation
disc = Discrete(env, 30)

# Allocate a Q table
q = Q(disc.size(), env.action_space.n)

# Learn!
learn_domain(env, q, disc, 500000)

In [ ]:
# Create a version of the domain with render_model = "human" so that you can watch it
env = gym.make(MOUNTAIN_CAR, render_mode="human")

# Observe the policy running
run_domain(env, q, disc, 500)

## Task 7: Experiment with all domaims

<b>You responses here will be part of your grade for this homework</b>

For each of the three domains:
* Describe the behavior in the domain when the action selection is random (i.e., when the Q-table is first initialized and prior to any training).  This can take the form of a few sentences that explain the behavior you are seeing and why random actions would lead to that behavior.
* Experiment with a few (3) different values of nbins when discretizing, some small and some larger, and explain differences in the learned policy as manifest when you run it in "human" mode between the different levels of descetization.  Again, describe the behavior you are seeing and why the level of discretization may have contributed to it, compared to the other behaviors you see for other levels of discretization.
* For one run in which you got the best learning results plot something that convinces me that learning occured.  That could be episode length through time (e.g., the faster the Mountain Car gets to the top of the hill the shorter the episodes) or reward through time (e.g., for the Lunar Lander).  Explain how the plot shows evidence that the system is learning.